# Lab04: TF-IDF.

- MSSV: 18600187
- Họ và tên: Vũ Cao Nguyên

## Yêu cầu bài tập

**Cách làm bài**


Bạn sẽ làm trực tiếp trên file notebook này; từ `TODO` cho biết những phần mà bạn cần phải làm.

Bạn có thể thảo luận ý tưởng cũng như tham khảo các tài liệu, nhưng *code và bài làm phải là của bạn*. 

Nếu vi phạm thì sẽ bị 0 điểm cho bài tập này.

**Cách nộp bài**

Trước khi nộp bài, rerun lại notebook (`Kernel` -> `Restart & Run All`).

Sau đó, tạo thư mục có tên `MSSV` của bạn (vd, nếu bạn có MSSV là 1234567 thì bạn đặt tên thư mục là `1234567`) Chép file notebook, file `tf_idf_data.txt` của các bạn (nếu file này kích thước lớn các bạn có thể chép link vào `link_data.txt`), nén thư mục `MSSV` này lại và nộp trên moodle.

**Nội dung bài tập**

Cài đặt một web crawler để thu thập và biểu diễn dữ liệu bằng không gian vector và trọng số TF-IDF: https://en.wikipedia.org/wiki/Web_mining.

## Nội dung bài tập

## 1. Không gian vector

- Một vector 2 chiều có thể được viết dưới dạng `[x,y]`
- Một vector 3 chiều có thể được viết dưới dạng `[x,y,z]`
![vector](vector.png)

- Đặt $V$ là tập hợp các đặc trưng trong thể hiện dữ liệu.
- Bất kỳ mẫu dữ liệu nào đều có thể được biểu diễn dưới dạng một vectơ với $\vert V\vert$ chiều
- Ví dụ: giả sử chúng ta có 3 đặc trưng là các từ dog, bite, man. Giá trị 1 thể hiện từ đó xuất hiện 1 lần, 0 là không xuất hiện.  
![space](space.png)


## 2. TF-IDF (Term Frequency - Inverse Document Frequency)
TF-IDF cho biết độ quan trọng của một từ đối với một tài liệu trong ngữ liệu.

TF- Term Frequency : dùng để ước lượng tần xuất xuất hiện của từ trong văn bản. Tuy nhiên với mỗi văn bản thì có độ dài khác nhau số lần xuất hiện của từ có thể nhiều hơn . Vì vậy số lần xuất hiện của từ sẽ được chuẩn hóa bằng cách chia cho độ dài của văn bản (tổng số từ trong văn bản đó).
    
$$tf_{t}=\dfrac{f(t,d)}{\sum_{i \in d}f(i,d)}$$ 

- $f(t,d)$ là số lần xuất hiện từ $t$ trong văn bản $d$.

IDF- Inverse Document Frequency: Khi tính tần số xuất hiện TF thì các từ đều được coi là quan trọng như nhau. Tuy nhiên có một số từ thường được được sử dụng nhiều nhưng không quan trọng để thể hiện ý nghĩa của đoạn văn như "is", "the"... (các từ chức năng). Ta cần giảm độ quan trọng của những từ này.

$$idf_t=\log \left(\dfrac{n}{df_t}\right)$$

- *n* là số văn bản.

- $df_t$ là số văn bản xuất hiện từ t

**TF-IDF:** $$tf_{t} \times idf_t$$

- $tf_{t}$ càng lớn tần suất xuất hiện của từ trong văn bản càng lớn.
- $idf_t$ càng lớn từ hiếm khi xuất hiện trong tập dữ liệu.
- **Giả định** những đặc trưng quan trọng nhất là những đặc trưng hiếm xuất hiện.

## 3. Thu thập và thể hiện dữ liệu
Thu thập dữ liệu bắt đầu từ URL: https://en.wikipedia.org/wiki/Web_mining.
- Gọi $D$ là một tập văn bản chứa *n* văn bản: $D=\left\{d_1,d_2,...,d_n\right\}$.
- $V=\left\{v_1,v_2,...v_{\vert V \vert}\right\}$ là từ điển (tất cả các từ phân biệt trong dữ liệu thu được). $\vert V \vert$ là kích thước từ điển.
- Một trọng số $w_{ij}$ được gán cho từ $t_i$ của văn bản dj thuộc $D$ xác định mức quan trọng của $t_i$ trong văn bản $d_j$. Từ không xuất hiện trong $d_j$ có $w_{ij}=0$.
- Mỗi văn bản $d_j$ được thể hiện dưới dạng vector $\mathbf{d_j}= [w_{1j},w_{2j},...,w_{\vert V \vert j}]$
- Thể hiện dữ liệu bằng một ma trận M kích thước $n \times \vert V \vert$ => mỗi hàng thể hiện một văn bản.

In [1]:
import requests
import re
from bs4 import BeautifulSoup
from bs4.element import Comment
import string
import pickle
import urllib.robotparser

rp = urllib.robotparser.RobotFileParser()
rp.set_url("https://en.wikipedia.org/robots.txt")
rp.read()
rrate = rp.request_rate("*")

rp.crawl_delay("*")

def get_urls(url):
    r = requests.get(url)
    urls = re.findall('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', r.text) 
    urls_copy = urls.copy()
    for url in urls_copy:
      if(rp.can_fetch("*", url) == False):
        urls.remove(url)
    return urls

def get_urls_recursive(start_url, limit):
    urls = get_urls(start_url)
    for url in urls:
        if(len(urls) >= limit):
            return urls[:limit]
        new_urls = get_urls(url)
        for new_url in new_urls:
          if(new_url not in urls and len(urls) < limit):
            urls.append(new_url)
    return urls[:limit]
url_list = get_urls_recursive('https://en.wikipedia.org/wiki/Web_mining', 3)
print((url_list))
print(len(url_list))

['https://en.wikipedia.org/wiki/Web_mining', 'https://archive.org/details/webinformationsy00ngua', 'https://archive.org/details/webinformationsy00ngua/page/n33']
3


In [2]:
def text_filter(element):
    if element.parent.name in ['style', 'title', 'script', 'head', '[document]', 'class', 'a', 'li']:
        return False
    elif isinstance(element, Comment):
        '''Opinion mining?'''
        return False
    elif re.match(r"[\s\r\n]+",str(element)): 
        '''space, return, endline'''
        return False
    return True 

In [3]:
def wordList(url):
    r=requests.get(url)
    soup=BeautifulSoup(r.content,"html.parser")
    text=soup.findAll(text=True)
    filtered_text=list(filter(text_filter,text))
    word_list=[]
    for astring in filtered_text:
      for types in string.punctuation:
        astring=astring.replace(types,' ')
        word_list.extend(astring.split(' '))
        word_list=[astr for astr in word_list if astr not in ' \n']
    return word_list

In [4]:
def read_url(url):
    word_list=wordList(url)
    result=[]
    for word in word_list:
      word = word.lower()
      if word not in result:
        result.append(word)
     
    return result 

In [5]:
data = set()
new_list=[];
for url in url_list:
    word_list = read_url(url)
    new_list.append(word_list)
    data = data.union(set(word_list))

In [6]:
import numpy as np
W = np.array(new_list, dtype=object)
print(W[0])

['web', 'mining', 'from', 'wikipedia,', 'the', 'free', 'encyclopedia', 'wikipedia', 'this', 'article', 'includes', 'a', ',', 'related', 'reading', 'or', 'but', 'its', 'sources', 'remain', 'unclear', 'because', 'it', 'lacks', '.', '(', 'october', '2020', ')', 'may', 'require', 'specific', 'problem', 'is:', 'is', 'needs', 'sufficient', 'inline', 'references,', 'and', 'complete', 're-write', 'has', 'been', 'proposed.', 'references', 're', 'write', 'proposed', 'uses', 'automated', 'methods', 'to', 'extract', 'both', 'structured', 'unstructured', 'data', 'pages,', 'server', 'logs', 'link', 'structures.', 'there', 'are', 'three', 'main', 'sub-categories', 'of', 'mining.', 'pages', 'sub', 'categories', 'structures', 'contents', '1', 'types', '2', 'usage', '2.1', 'pros', '2.2', 'cons', '3', 'structure', '4', 'content', '4.1', 'in', 'foreign', 'languages', '4.1.1', 'chinese', '5', 'see', 'also', '6', '6.1', 'books', '6.2', 'bibliographic', '[', ']', 'can', 'be', 'divided', 'into', 'different', 

In [7]:
def count(words):
  dictionary = dict.fromkeys(data, 0)
  for word in words:
    dictionary[word]+=1
  return dictionary

In [8]:
# TF
def calculateTF(dictionary, words):
    tfDictionary = {}
    wordsCount = len(words)
    for word, count in dictionary.items():
        tfDictionary[word] = count/float(wordsCount)
    return tfDictionary

In [9]:
# IDF
def calculateIDF(document_list):
    import math
    idfDictionary = {}
    N = len(document_list) 
    idfDictionary = dict.fromkeys(document_list[0].keys(), 0)
    for doc in document_list:
        for word, val in doc.items():
            if val > 0:
                idfDictionary[word] += 1
    for word, val in idfDictionary.items():
        idfDictionary[word] = math.log10(N / float(val))
        
    return idfDictionary

In [10]:
def calculateTFxIDF(tfs, idfs):
    tfxidf = {}
    for word, val in tfs.items():
        tfxidf[word] = val*idfs[word]
    return tfxidf

In [11]:
listTF=[];
for words in W:
  wordConut=count(words)
  tfdoc = calculateTF(wordConut, words)
  listTF.append(tfdoc)

In [12]:
idfs = calculateIDF(listTF)
print(idfs)

{'semantic': 0.17609125905568124, 'related-external-id': 0.17609125905568124, 'engineering,': 0.17609125905568124, 'navigation': 0.47712125471966244, ':': 0.0, 'sheng,': 0.47712125471966244, 'on': 0.0, 'book': 0.17609125905568124, 'pa:': 0.47712125471966244, 'ngu,': 0.47712125471966244, 'short': 0.17609125905568124, 'pernul,': 0.47712125471966244, 'dhakal,': 0.47712125471966244, ',': 0.0, 'explorations': 0.47712125471966244, 'provide': 0.47712125471966244, 'invaded.': 0.47712125471966244, 'categorize,': 0.47712125471966244, 'reddit': 0.17609125905568124, 'vries': 0.47712125471966244, 'without': 0.47712125471966244, 'methodology': 0.17609125905568124, 'index\n': 0.17609125905568124, 'façade': 0.17609125905568124, 'topical': 0.17609125905568124, 'mining:': 0.47712125471966244, 'category': 0.47712125471966244, 'e-commerce,': 0.17609125905568124, '2007).': 0.47712125471966244, '07': 0.17609125905568124, 'it': 0.47712125471966244, '18': 0.17609125905568124, 'responsible': 0.4771212547196624

In [13]:
sum_list=[]
for tfs in listTF:
  tfxidf = calculateTFxIDF(tfs, idfs)
  print(tfxidf)
  sum_list.append(tfxidf)


{'semantic': 0.0, 'related-external-id': 0.0, 'engineering,': 0.0, 'navigation': 0.0008100530640401739, ':': 0.0, 'sheng,': 0.0008100530640401739, 'on': 0.0, 'book': 0.0, 'pa:': 0.0008100530640401739, 'ngu,': 0.0008100530640401739, 'short': 0.0, 'pernul,': 0.0008100530640401739, 'dhakal,': 0.0008100530640401739, ',': 0.0, 'explorations': 0.0008100530640401739, 'provide': 0.0008100530640401739, 'invaded.': 0.0008100530640401739, 'categorize,': 0.0008100530640401739, 'reddit': 0.0, 'vries': 0.0008100530640401739, 'without': 0.0008100530640401739, 'methodology': 0.0, 'index\n': 0.0, 'façade': 0.0, 'topical': 0.0, 'mining:': 0.0008100530640401739, 'category': 0.0008100530640401739, 'e-commerce,': 0.0, '2007).': 0.0008100530640401739, '07': 0.0, 'it': 0.0008100530640401739, '18': 0.0, 'responsible': 0.0008100530640401739, 'toggled': 0.0, 'interpret': 0.0008100530640401739, 'of': 0.0, 'contribute': 0.0008100530640401739, 'factors': 0.0008100530640401739, '|journal=': 0.0008100530640401739, '

In [14]:
import numpy as np
M=np.zeros(shape=(len(sum_list),len(sum_list[0])),dtype=float)
i = 0
for sum in sum_list:
  j=0
  for word, val in sum.items():
    M[i][j]=val
    j+=1
  i+=1
print(M)

[[0.         0.         0.         ... 0.         0.00081005 0.        ]
 [0.00047981 0.00047981 0.00047981 ... 0.00047981 0.         0.00047981]
 [0.00047981 0.00047981 0.00047981 ... 0.00047981 0.         0.00047981]]


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
np.savetxt("/content/drive/My Drive/Colab Notebooks/18600187.txt", M, fmt="%s")